---
**Import packages**

In [125]:
import json
import pandas as pd

----
**Fetching data**

In [126]:
df = pd.read_excel("../data/raw_data.xlsx")
df.head()

,id,comm_type,raw_content,source_id,ingested_at,processed_at,is_processed,subject
0,de819d74-9b7d-4b60-863c-5f7f634a6402,call,"{""id"": ""DDCD444CF2A04CA1BA108EE5"", ""title"": ""U...",32A19D011A4C41F09FE3A2FB,2025-06-12T13:29:09,2025-03-22T11:37:50,True,Profit-focused composite time-frame
1,d4746716-3350-4e7f-952f-b39a8dc430d3,call,"{""id"": ""E462A7000DCA4A688FB880B3"", ""title"": ""E...",485F87C4F6E34F3189D1E3E9,2025-03-11T19:29:43,2025-03-06T01:09:40,True,Ergonomic dedicated process improvement
2,e509a9f2-c9a0-4ad0-9489-e22825e40488,meeting,"{""id"": ""66DE26DE41BC462AAE1BA7C8"", ""title"": ""S...",B7E98770F4A542729D65AF41,2025-03-27T06:41:29,2025-01-12T01:51:05,True,Focused clear-thinking parallelism
3,77fb5908-4181-45b8-b484-f7bd4dd294c9,meeting,"{""id"": ""69D47DC9F39A4AA29B34AEFD"", ""title"": ""S...",7A00DCB707B74395B2DD552F,2025-05-06T02:23:25,2025-03-04T16:29:27,True,Re-engineered object-oriented emulation
4,6ba3fa0e-41c1-4ed6-a782-900cc6af3580,call,"{""id"": ""0D1BABD93ED04F099F2BCE2D"", ""title"": ""S...",1C96310A145B445799924A8F,2025-05-09T05:48:27,2025-05-07T15:35:37,True,Focused 24hour policy


---
**Analyse Data**

In [127]:
df.isna().sum()

id              0
comm_type       0
raw_content     0
source_id       0
ingested_at     0
processed_at    0
is_processed    0
subject         0
dtype: int64

In [128]:
columns = df.columns
columns

Index(['id', 'comm_type', 'raw_content', 'source_id', 'ingested_at',
       'processed_at', 'is_processed', 'subject'],
      dtype='object')

---
**Cleaning raw_content**

In [129]:
json_data = df['raw_content']
data = []

decoder = json.JSONDecoder()

for line in json_data:
    s = line.strip()
    while s:
        try:
            obj, idx = decoder.raw_decode(s)
            data.append(obj)
            s = s[idx:].lstrip()
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            break

JSON decode error: Expecting value: line 1 column 1 (char 0)


In [130]:
def json_to_df(item: dict):
    content_data = {
        'id': item['id'],
        'title': item['title'],
        'duration': item['duration'],
        'audio_url': item['audio_url'],
        'video_url': item['video_url'],
        'date_string': item['dateString'],
        'host_email': item['host_email'],
        'calendar': item['calendar_id'],
        'transcript_url': item['transcript_url'],
        'organizer_email': item['organizer_email']
    }
    df_content = pd.DataFrame([content_data])

    df_speakers = pd.DataFrame(item['speakers'])
    df_speakers['content_id'] = item['id']

    df_participants = pd.DataFrame(item['participants'], columns=['email'])
    df_participants['content_id'] = item['id']

    df_attendees = pd.DataFrame(item['meeting_attendees'])
    df_attendees['content_id'] = item['id']

    return df_content, df_speakers, df_participants,df_attendees

---
**Seperate table from column**

In [131]:
def seperate(table, column):
    values = table[column].dropna().unique()
    new_table = pd.DataFrame({
        'id': range(1, len(values) + 1),
        'name': values
    })
    map_table = dict(zip(new_table['name'], new_table['id']))
    table[column] = table[column].map(map_table)
    return new_table, table

---
**Split data to tables**

In [132]:
Subject = df['subject'].dropna().unique()
CommunicationType = df['comm_type'].dropna().unique()

In [134]:
CommunicationType = pd.DataFrame(CommunicationType, columns=['comm_type'])
Subject = pd.DataFrame(Subject, columns=['name'])

In [135]:
comm_type, table = seperate(df, 'comm_type')
comm_type

,id,name
0,1,call
1,2,meeting


In [136]:
subject, table = seperate(df, 'subject')
subject

,id,name
0,1,Profit-focused composite time-frame
1,2,Ergonomic dedicated process improvement
2,3,Focused clear-thinking parallelism
3,4,Re-engineered object-oriented emulation
4,5,Focused 24hour policy
5,6,Realigned homogeneous circuit
6,7,Advanced cohesive extranet
7,8,Advanced needs-based product
8,9,Right-sized hybrid Graphic Interface
9,10,Right-sized scalable moratorium


In [137]:
content_ids = [item['id'] for item in data]
df['content_id'] = content_ids

In [138]:
Communication = df[['id','comm_type', 'source_id', 'ingested_at', 'processed_at', 'is_processed', 'content_id', 'subject']]

In [140]:
all_content = []
all_speakers = []
all_participants = []
all_attendees = []

for item in data:
    df_content, df_speakers, df_participants, df_attendees = json_to_df(item)
    all_content.append(df_content)
    all_speakers.append(df_speakers)
    all_participants.append(df_participants)
    all_attendees.append(df_attendees)

df_all_content = pd.concat(all_content, ignore_index=True)
df_all_speakers = pd.concat(all_speakers, ignore_index=True)
df_all_participants = pd.concat(all_participants, ignore_index=True)
df_all_attendees = pd.concat(all_attendees, ignore_index=True)


---
**Database**

In [141]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-L0MHKPI\\SQLEXPRESS;"
    "DATABASE=data_tranformation;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()


In [142]:
for _, row in df_all_content.iterrows():
    cursor.execute("""
        INSERT INTO Content (
            ID, Title, Duration, AudioUrl, VideoUrl, Date, HostEmail,
            CalendarRef, TranscriptUrl, OrganizerEmail
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, row['id'], row['title'], row['duration'], row['audio_url'], row['video_url'],
            row['date_string'], row['host_email'], row['calendar'],
            row['transcript_url'], row['organizer_email'])

In [143]:
for _, row in df_all_speakers.iterrows():
    cursor.execute("""
        INSERT INTO Speaker (Name, ContentID) VALUES (?, ?)
    """, row['name'], row['content_id'])

In [144]:
for _, row in df_all_participants.iterrows():
    cursor.execute("""
        INSERT INTO Participant (Email, ContentID) VALUES (?, ?)
    """, row['email'], row['content_id'])


In [145]:
for _, row in df_all_attendees.iterrows():
    cursor.execute("""
        INSERT INTO MeetingAttendees (
            Name, Email, Location, DisplayName, PhoneNumber, ContentID
        ) VALUES (?, ?, ?, ?, ?, ?)
    """, row['name'], row['email'], row['location'],
         row['displayName'], row['phoneNumber'], row['content_id'])


In [146]:
for _, row in CommunicationType.iterrows():
    cursor.execute("""
        INSERT INTO CommunicationTypes (Name) VALUES (?)
    """, row['comm_type'])

In [147]:
for _, row in Subject.iterrows():
    cursor.execute("""
        INSERT INTO Subject (Name) VALUES (?)
    """, row['name'])

In [148]:
for _, row in Communication.iterrows():
    cursor.execute("""
        INSERT INTO Communication (
            ID, CommunicationTypeID, ContentID, SourceID, IngestedAt, ProcessedAt, IsProcessed, SubjectID
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, row['id'], row['comm_type'], row['content_id'],
         row['source_id'], row['ingested_at'],
         row['processed_at'], row['is_processed'], row['subject'])

In [149]:
conn.commit()
cursor.close()
conn.close()